# Vaccinations Administered by County

In [ ]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from api_keys import geoapify_key
from api_keys import weather_api_key
from scipy import stats

In [ ]:
# API
url = "http://api.openweathermap.org/data/2.5/weather?"
query_url = f"{url}q={county}&units=metric&appid={weather_api_key}"
url_vaccine = "https://data.cdc.gov/resource/8xkx-amqh.json?$query=SELECT%0A%20%20%60date%60%2C%0A%20%20%60fips%60%2C%0A%20%20%60mmwr_week%60%2C%0A%20%20%60recip_county%60%2C%0A%20%20%60recip_state%60%2C%0A%20%20%60completeness_pct%60%2C%0A%20%20%60administered_dose1_recip%60%2C%0A%20%20%60administered_dose1_pop_pct%60%2C%0A%20%20%60administered_dose1_recip_5plus%60%2C%0A%20%20%60administered_dose1_recip_5pluspop_pct%60%2C%0A%20%20%60administered_dose1_recip_12plus%60%2C%0A%20%20%60administered_dose1_recip_12pluspop_pct%60%2C%0A%20%20%60administered_dose1_recip_18plus%60%2C%0A%20%20%60administered_dose1_recip_18pluspop_pct%60%2C%0A%20%20%60administered_dose1_recip_65plus%60%2C%0A%20%20%60administered_dose1_recip_65pluspop_pct%60%2C%0A%20%20%60series_complete_yes%60%2C%0A%20%20%60series_complete_pop_pct%60%2C%0A%20%20%60series_complete_5plus%60%2C%0A%20%20%60series_complete_5pluspop_pct%60%2C%0A%20%20%60series_complete_5to17%60%2C%0A%20%20%60series_complete_5to17pop_pct%60%2C%0A%20%20%60series_complete_12plus%60%2C%0A%20%20%60series_complete_12pluspop_pct%60%2C%0A%20%20%60series_complete_18plus%60%2C%0A%20%20%60series_complete_18pluspop_pct%60%2C%0A%20%20%60series_complete_65plus%60%2C%0A%20%20%60series_complete_65pluspop_pct%60%2C%0A%20%20%60booster_doses%60%2C%0A%20%20%60booster_doses_vax_pct%60%2C%0A%20%20%60booster_doses_5plus%60%2C%0A%20%20%60booster_doses_5plus_vax_pct%60%2C%0A%20%20%60booster_doses_12plus%60%2C%0A%20%20%60booster_doses_12plus_vax_pct%60%2C%0A%20%20%60booster_doses_18plus%60%2C%0A%20%20%60booster_doses_18plus_vax_pct%60%2C%0A%20%20%60booster_doses_50plus%60%2C%0A%20%20%60booster_doses_50plus_vax_pct%60%2C%0A%20%20%60booster_doses_65plus%60%2C%0A%20%20%60booster_doses_65plus_vax_pct%60%2C%0A%20%20%60second_booster_50plus%60%2C%0A%20%20%60second_booster_50plus_vax_pct%60%2C%0A%20%20%60second_booster_65plus%60%2C%0A%20%20%60second_booster_65plus_vax_pct%60%2C%0A%20%20%60svi_ctgy%60%2C%0A%20%20%60series_complete_pop_pct_svi%60%2C%0A%20%20%60series_complete_5pluspop_pct_svi%60%2C%0A%20%20%60series_complete_5to17pop_pct_svi%60%2C%0A%20%20%60series_complete_12pluspop_pct_svi%60%2C%0A%20%20%60series_complete_18pluspop_pct_svi%60%2C%0A%20%20%60series_complete_65pluspop_pct_svi%60%2C%0A%20%20%60metro_status%60%2C%0A%20%20%60series_complete_pop_pct_ur_equity%60%2C%0A%20%20%60series_complete_5pluspop_pct_ur_equity%60%2C%0A%20%20%60series_complete_5to17pop_pct_ur_equity%60%2C%0A%20%20%60series_complete_12pluspop_pct_ur_equity%60%2C%0A%20%20%60series_complete_18pluspop_pct_ur_equity%60%2C%0A%20%20%60series_complete_65pluspop_pct_ur_equity%60%2C%0A%20%20%60booster_doses_vax_pct_svi%60%2C%0A%20%20%60booster_doses_12plusvax_pct_svi%60%2C%0A%20%20%60booster_doses_18plusvax_pct_svi%60%2C%0A%20%20%60booster_doses_65plusvax_pct_svi%60%2C%0A%20%20%60booster_doses_vax_pct_ur_equity%60%2C%0A%20%20%60booster_doses_12plusvax_pct_ur_equity%60%2C%0A%20%20%60booster_doses_18plusvax_pct_ur_equity%60%2C%0A%20%20%60booster_doses_65plusvax_pct_ur_equity%60%2C%0A%20%20%60census2019%60%2C%0A%20%20%60census2019_5pluspop%60%2C%0A%20%20%60census2019_5to17pop%60%2C%0A%20%20%60census2019_12pluspop%60%2C%0A%20%20%60census2019_18pluspop%60%2C%0A%20%20%60census2019_65pluspop%60%2C%0A%20%20%60bivalent_booster_5plus%60%2C%0A%20%20%60bivalent_booster_5plus_pop_pct%60%2C%0A%20%20%60bivalent_booster_12plus%60%2C%0A%20%20%60bivalent_booster_12plus_pop_pct%60%2C%0A%20%20%60bivalent_booster_18plus%60%2C%0A%20%20%60bivalent_booster_18plus_pop_pct%60%2C%0A%20%20%60bivalent_booster_65plus%60%2C%0A%20%20%60bivalent_booster_65plus_pop_pct%60"


In [ ]:
# Clean data

# Want columns 
#Recip_County
#Administered_Dose1_Recip_5Plus, Administered_Dose1_Recip_12Plus,
#Administered_Dose1_Recip_18Plus, Administered_Dose1_Recip_65Plus,
#Add these four columns together for total dose 1 administered

#Series_Complete_5Plus, Series_Complete_12Plus, Series_Complete_18Plus, Series_Complete_65Plus
#Add these four columns together for total series administrated



In [ ]:
# Find latitude and longitude for each county
countys = [len("Recip_County")]

lat = []
lon = []

for county in countys:
    response = requests.get(query_url + county).json()
    lat.append['coord']['lat']
    lon.append['coord']['lon']
    


In [ ]:
# Drop columns with null data


In [1]:
# Create dataframe
vaccine_df = pd.DataFrame({
    "Recip_County": countys,
    "lat": lat,
    "lon": lon,
    "Administered_Dose1_Recip_5Plus": , 
    "Administered_Dose1_Recip_12Plus": ,
    "Administered_Dose1_Recip_18Plus": , 
    "Administered_Dose1_Recip_65Plus": ,
    "Series_Complete_5Plus": , 
    "Series_Complete_12Plus": , 
    "Series_Complete_18Plus": , 
    "Series_Complete_65Plus": 
    
})

#Rename columns


vaccine_df = pd.DataFrame(vaccine_dict)
vaccine_df

SyntaxError: expression expected after dictionary key and ':' (947557971.py, line 4)

In [ ]:
# Statistical summary for 1 dose
#Alter once totals for vaccinations of dose 1 and series calculated
mean_dose1_5 = vaccine_df['Administered_Dose1_Recip_5Plus'].groupby(vaccine_df['County']).mean()
median_dose1_5 = vaccine_df['Administered_Dose1_Recip_5Plus'].groupby(vaccine_df['County']).median()
variance_dose1_5 = vaccine_study_df['Administered_Dose1_Recip_5Plus'].groupby(vaccine_df['County']).var()
std_dev_dose1_5 = vaccine_df['Administered_Dose1_Recip_5Plus'].groupby(vaccine_df['County']).std()
sem_dose1_5 = vaccine_df['Administered_Dose1_Recip_5Plus'].groupby(vaccine_df['County']).sem()

mean_dose1_12 = vaccine_df['Administered_Dose1_Recip_12Plus'].groupby(vaccine_df['County']).mean()
median_dose1_12 = vaccine_df['Administered_Dose1_Recip_12Plus'].groupby(vaccine_df['County']).median()
variance_dose1_12 = vaccine_df['Administered_Dose1_Recip_12Plus'].groupby(vaccine_df['County']).var()
std_dev_dose1_12 = vaccine_df['Administered_Dose1_Recip_12Plus'].groupby(vaccine_df['County']).std()
sem_dose1_12 = vaccine_df['Administered_Dose1_Recip_12Plus'].groupby(vaccine_df['County']).sem()

mean_dose1_18 = vaccine_df['Administered_Dose1_Recip_18Plus'].groupby(vaccine_df['County']).mean()
median_dose1_18 = vaccine_df['Administered_Dose1_Recip_18Plus'].groupby(vaccine_df['County']).median()
variance_dose1_18 = vaccine_df['Administered_Dose1_Recip_18Plus'].groupby(vaccine_df['County']).var()
std_dev_dose1_18 = vaccine_df['Administered_Dose1_Recip_18Plus'].groupby(vaccine_df['County']).std()
sem_dose1_18 = vaccine_df['Administered_Dose1_Recip_18Plus'].groupby(vaccine_df['County']).sem()

mean_dose1_65 = vaccine_df['Administered_Dose1_Recip_65Plus'].groupby(vaccine_df['County']).mean()
median_dose1_65 = vaccine_df['Administered_Dose1_Recip_65Plus'].groupby(vaccine_df['County']).median()
variance_dose1_65 = vaccine_df['Administered_Dose1_Recip_65Plus'].groupby(vaccine_df['County']).var()
std_dev_dose1_65 = vaccine_df['Administered_Dose1_Recip_65Plus'].groupby(vaccine_df['County']).std()
sem_dose1_65 = vaccine_df['Administered_Dose1_Recip_65Plus'].groupby(vaccine_df['County']).sem()

summary_dose1_df = pd.DataFrame({"Mean Dose 1 Vaccine Distribution (Age 5+)": mean_dose1_5,
                           "Mean Dose 1 Vaccine Distribution (Age 12+)": mean_dose1_12,
                           "Mean Dose 1 Vaccine Distribution (Age 18+)": mean_dose1_18,
                           "Mean Dose 1 Vaccine Distribution (Age 65+)": mean_dose1_65,
                           "Median Dose 1 Vaccine Distribution (Age 5+)": median_dose1_5,
                           "Median Dose 1 Vaccine Distribution (Age 12+)": median_dose1_12,
                           "Median Dose 1 Vaccine Distribution (Age 18+)": median_dose1_18,
                           "Median Dose 1 Vaccine Distribution (Age 65+)": median_dose1_65,
                           "Dose 1 Vaccine Distribution Variance (Age 5+)": variance_dose1_5,
                           "Dose 1 Vaccine Distribution Variance (Age 12+)": variance_dose1_12,
                           "Dose 1 Vaccine Distribution Variance (Age 18+)": variance_dose1_18,
                           "Dose 1 Vaccine Distribution Variance (Age 65+)": variance_dose1_65,
                           "Dose 1 Vaccine Distribution Std. Dev (Age 5+)": std_dev_dose1_5,
                           "Dose 1 Vaccine Distribution Std. Dev (Age 12+)": std_dev_dose1_12,
                           "Dose 1 Vaccine Distribution Std. Dev (Age 18+)": std_dev_dose1_18,
                           "Dose 1 Vaccine Distribution Std. Dev (Age 65+)": std_dev_dose1_65,
                           "Dose 1 Vaccine Distribution Std. Err. (Age 5+)": sem_dose1_5,
                           "Dose 1 Vaccine Distribution Std. Err. (Age 12+)": sem_dose1_12,
                           "Dose 1 Vaccine Distribution Std. Err. (Age 18+)": sem_dose1_18,
                           "Dose 1 Vaccine Distribution Std. Err. (Age 65+)": sem_dose1_65})
summary_dose1_df

In [ ]:
# Statistical summary for series
mean_series_5 = vaccine_df['Series_Complete_5Plus'].groupby(vaccine_df['County']).mean()
median_series_5 = vaccine_df['Series_Complete_5Plus'].groupby(vaccine_df['County']).median()
variance_series_5 = vaccine_df['Series_Complete_5Plus'].groupby(vaccine_df['County']).var()
std_dev_series_5 = vaccine_df['Series_Complete_5Plus'].groupby(vaccine_df['County']).std()
sem_series_5 = vaccine_df['Series_Complete_5Plus'].groupby(vaccine_df['County']).sem()

mean_series_12 = vaccine_df['Series_Complete_12Plus'].groupby(vaccine_df['County']).mean()
median_series_12 = vaccine_df['Series_Complete_12Plus'].groupby(vaccine_df['County']).median()
variance_series_12 = vaccine_df['Series_Complete_12Plus'].groupby(vaccine_df['County']).var()
std_dev_series_12 = vaccine_df['Series_Complete_12Plus'].groupby(vaccine_df['County']).std()
sem_series_12 = vaccine_df['Series_Complete_12Plus'].groupby(vaccine_df['County']).sem()

mean_series_18 = vaccine_df['Series_Complete_18Plus'].groupby(vaccine_df['County']).mean()
median_series_18 = vaccine_df['Series_Complete_18Plus'].groupby(vaccine_df['County']).median()
variance_series_18 = vaccine_df['Series_Complete_18Plus'].groupby(vaccine_df['County']).var()
std_dev_series_18 = vaccine_df['Series_Complete_18Plus'].groupby(vaccine_df['County']).std()
sem_series_18 = vaccine_df['Series_Complete_18Plus'].groupby(vaccine_df['County']).sem()

mean_series_65 = vaccine_df['Series_Complete_65Plus'].groupby(vaccine_df['County']).mean()
median_series_65 = vaccine_df['Series_Complete_65Plus'].groupby(vaccine_df['County']).median()
variance_series_65 = vaccine_df['Series_Complete_65Plus'].groupby(vaccine_df['County']).var()
std_dev_series_65 = vaccine_df['Series_Complete_65Plus'].groupby(vaccine_df['County']).std()
sem_series_65 = vaccine_df['Series_Complete_65Plus'].groupby(vaccine_df['County']).sem()

summary_series_df = pd.DataFrame({"Mean Series Vaccine Distribution (Age 5+)": mean_series_5,
                                  "Mean Series Vaccine Distribution (Age 12+)": mean_series_12,
                                  "Mean Series Vaccine Distribution (Age 18+)": mean_series_18,
                                  "Mean Series Vaccine Distribution (Age 65+)": mean_series_65,
                                  "Median Series Vaccine Distribution (Age 5+)": median_series_5,
                                  "Median Series Vaccine Distribution (Age 12+)": median_series_12,
                                  "Median Series Vaccine Distribution (Age 18+)": median_series_18,
                                  "Median Series Vaccine Distribution (Age 65+)": median_series_65,
                                  "Series Vaccine Distribution Variance (Age 5+)" variance_series_5,
                                  "Series Vaccine Distribution Variance (Age 12+)" variance_series_12,
                                  "Series Vaccine Distribution Variance (Age 18+)" variance_series_18,
                                  "Series Vaccine Distribution Variance (Age 65+)" variance_series_65,
                                  "Series Vaccine Distribution Std. Dev (Age 5+)": std_dev_series_5,
                                  "Series Vaccine Distribution Std. Dev (Age 12+)": std_dev_series_12,
                                  "Series Vaccine Distribution Std. Dev (Age 18+)": std_dev_series_18,
                                  "Series Vaccine Distribution Std. Dev (Age 65+)": std_dev_series_65,
                                  "Series Vaccine Distribution Std. Err (Age 5+)": sem_series_5,
                                  "Series Vaccine Distribution Std. Err (Age 12+)": sem_series_12,
                                  "Series Vaccine Distribution Std. Err (Age 18+)": sem_series_18,
                                  "Series Vaccine Distribution Std. Err (Age 65+)": sem_series_65})
summary_series_df

In [ ]:
# Configure map
map_plot_1 = airports_df.hvplot.points(
    "Lon",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Number of Vaccines Administrated",
    scale = 0.5, 
    color = "County"
)

# Display the map plot
map_plot_1